Comparing the compute time for running the interior-point method on three different formulations of the optimization problem: (1) dual, (2) primal, and (3) modified primal.

In [16]:
using RCall
using Mosek
using JuMP
using PyPlot
using Distributions
using LowRankApprox

In [33]:
include("../code/julia/datasim.jl");
include("../code/julia/likelihood.jl");
include("../code/julia/mixSQP_time.jl");
include("../code/julia/primaldual.jl");
include("../code/julia/REBayes.jl");

In [28]:
srand(1);
n = round(Int,1e4); m = 40;
x = normtmixdatasim(n);
sd = autoselectmixsd(x,nv = m);
L  = normlikmatrix(x,sd = sd);
mixSQP_time(L);

In [59]:
n = round(Int,1e4); m = 40;
x = normtmixdatasim(n);
sd = autoselectmixsd(x,nv = m);
L  = normlikmatrix(x,sd = sd);
@time ip_dual(L);
@time ip_simp(L);
@time ip_box(L);
@time sqp_dual(L);
@time sqp_simp(L);
@time sqp_box(L);
@time REBayes(L);

  0.787168 seconds (1.47 M allocations: 192.761 MiB, 6.36% gc time)
  5.001482 seconds (3.85 M allocations: 515.166 MiB, 10.92% gc time)
  4.685277 seconds (3.85 M allocations: 515.152 MiB, 8.50% gc time)
 39.504578 seconds (124.12 M allocations: 12.858 GiB, 25.09% gc time)
  0.277916 seconds (10.46 k allocations: 255.354 MiB, 12.93% gc time)
  0.034491 seconds (1.65 k allocations: 50.891 MiB, 15.50% gc time)
  0.413143 seconds (217 allocations: 9.797 KiB)
